In [1]:
import gym
import numpy as np
import random
import time
from IPython.display import clear_output
import tensorflow as tf

In [2]:
env = gym.make("FrozenLake-v0")
action_space_size = env.action_space.n
state_space_size = env.observation_space.n
q_table = np.zeros((state_space_size, action_space_size))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [8]:
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

rewards_all_episodes = []
# Q-Learning Algorithm
config = tf.ConfigProto(log_device_placement = True)
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
with tf.device('/gpu:0'):
    for episode in range(num_episodes):
        state = env.reset()
        done = False
        rewards_current_episode = 0

        for step in range(max_steps_per_episode):
            #Exploration-exploitation trade-off
            exploration_rate_threshold = random.uniform(0,1)
            if exploration_rate_threshold > exploration_rate:
                action = np.argmax(q_table[state,:])
            else:
                action = env.action_space.sample()
            new_state,reward, done,info = env.step(action)

            #Update Q-table for Q(s,a)
            q_table[state, action] = q_table[state, action]*(1-learning_rate) + \
                                     (reward + \
                                      discount_rate*np.max(q_table[new_state,:]))*learning_rate
            state = new_state
            rewards_current_episode += reward
            if done == True:
                break
        #exploration Rate Decay
        exploration_rate = min_exploration_rate + \
                       (max_exploration_rate - min_exploration_rate)*np.exp(-exploration_decay_rate*episode)      
        rewards_all_episodes.append(rewards_current_episode)


In [9]:
#After All Episodes complete
#Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes//1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.05000000000000004
2000 :  0.19900000000000015
3000 :  0.4150000000000003
4000 :  0.5620000000000004
5000 :  0.6330000000000005
6000 :  0.6500000000000005
7000 :  0.6630000000000005
8000 :  0.6790000000000005
9000 :  0.6800000000000005
10000 :  0.6720000000000005


In [12]:
#Agent Plays a game
for episode in range(3):
    state = env.reset()
    done = False
    print(f"  EPISODE  ", {episode+1},"\n\n\n")
    time.sleep(1)

    for step in range(max_steps_per_episode):
        clear_output(wait = True)
        env.render()
        time.sleep(0.3)

        action = np.argmax(q_table[state,:])
        new_state, reward, done, info = env.step(action)
        if done:
            clear_output(wait = True)
            env.render()
            if reward == 1:
                print("Goal Reached!")
                time.sleep(3)
            else:
                print("Fell through the Hole")
                time.sleep(3)
            clear_output(wait = True)
            break
        state = new_state
env.close()



            
        

  (Down)
SFFF
FHFH
FFFH
HFFG
Goal Reached!


In [13]:
print(f'Updated Q_table\n{q_table}') 

Updated Q_table
[[0.5460029  0.50569908 0.49721488 0.52010591]
 [0.39821809 0.37491592 0.31487093 0.50198887]
 [0.41913103 0.39427812 0.40887079 0.45406792]
 [0.27189729 0.28161037 0.32598263 0.43250466]
 [0.57625074 0.42028216 0.30637439 0.37377493]
 [0.         0.         0.         0.        ]
 [0.17285462 0.19540166 0.35767125 0.13136019]
 [0.         0.         0.         0.        ]
 [0.40490916 0.42927755 0.29248408 0.61339724]
 [0.44893081 0.64081063 0.47324999 0.38166263]
 [0.58804203 0.49556487 0.39712862 0.3943681 ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.46362698 0.4016639  0.75571435 0.42862467]
 [0.70602968 0.86752954 0.71395524 0.74970005]
 [0.         0.         0.         0.        ]]


'/device:GPU:0'